## Browser Automation and Scraping: CBP Wait Times

### The Objective:
#### Web scraping and data formatting are two of the most fundamental skills taught at Columbia University's [Lede Program](https://ledeprogram.com/) for data journalism. 
#### ...and properly presenting Python-based code via publishing tools like Quarto is another important objective. 


### The Tools and Processes: 
#### Scraping the US [Customs and Border Protection site](https://awt.cbp.gov/), obtaining information, formatting that information, feeding it to a chart-making platform -- and making it automatic. 
1. Browser automation implemented with Playwright
2. Web scraping performed with Beautiful Soup4
3. Charts automatically generated by Datawrapper...
4. Same process repeated every day at 1300 GMT thanks to GitHub Actions



#### import all relevant libraries and modules: 

In [11]:
from playwright.async_api import async_playwright
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime
import time
import json
import csv
import numpy as np
import re
import lxml.html

#### Use Playwright to launch the Chromium browser

In [12]:
playwright = await async_playwright().start()

In [13]:
browser = await playwright.chromium.launch(headless = True)

In [14]:
page = await browser.new_page()

#### ... and open the CBP webpage

In [15]:
await page.goto("https://awt.cbp.gov/")
time.sleep(6)

### Use Playwright to pick an airport -- JFK in our case: 

In [16]:
await page.get_by_label("Select Your Airport/Terminal").select_option("JFK")

['JFK']

#### and also have Playwright click on the "Create Report" button: 

In [17]:
await page.get_by_role("button", name="Create Report").click()
time.sleep(10)

In [18]:
page = await page.content()
tables = pd.read_html(page)
tables[0]

Airport               Terminal      Date         Hour       U.S. Citizen  \
   Airport               Terminal      Date         Hour       U.S. Citizen   
   Airport               Terminal      Date         Hour Average Wait  Time   
0      JFK             Terminal 1  8/7/2023  1000 - 1100                 32   
1      JFK             Terminal 1  8/7/2023  1100 - 1200                 41   
2      JFK             Terminal 1  8/7/2023  1300 - 1400                 31   
3      JFK             Terminal 1  8/7/2023  1400 - 1500                 41   
4      JFK             Terminal 1  8/7/2023  1500 - 1600                 61   
..     ...                    ...       ...          ...                ...   
71     JFK  Terminal 8 (American)  8/7/2023  1900 - 2000                 34   
72     JFK  Terminal 8 (American)  8/7/2023  2000 - 2100                 32   
73     JFK  Terminal 8 (American)  8/7/2023  2100 - 2200                 26   
74     JFK  Terminal 8 (American)  8/7/2023  2200 - 2300                 13   
75     JFK  Terminal 8 (American)  8/7/2023  2300 - 0000                 21   

                    Non U.S. Citizen                               All  \
                    Non U.S. Citizen                        Wait Times   
   Max Wait  Time Average Wait  Time Max Wait  Time Average Wait  Time   
0              53                 29             52                 30   
1              91                 90            165                 72   
2             102                 97            153                 73   
3             138                110            179                 94   
4             148                134            171                107   
..            ...                ...            ...                ...   
71            115                 65            116                 46   
72             70                 68            102                 49   
73             63                 55             69                 43   
74             55                 34             94                 25   
75             89                 71             97                 42   

                   ...                                                       \
                   ... Number Of Passengers Time Interval                     
   Max Wait  Time  ...                              16-30 31-45 46-60 61-90   
0              53  ...                                146   270    82     0   
1             165  ...                                207   319   208   300   
2             153  ...                                211   322    64    61   
3             179  ...                                 87   204    77    35   
4             171  ...                                 37    57    74   120   
..            ...  ...                                ...   ...   ...   ...   
71            116  ...                                194   176   161   239   
72            102  ...                                 38   264   118   161   
73             69  ...                                 18    67    47    67   
74             94  ...                                329   169    63    96   
75             97  ...                                164    59    11   139   

                                                            \
                   Unnamed: 17_level_1 Unnamed: 18_level_1   
   91-120 120 plus            Excluded               Total   
0       0        0                  19                 624   
1     206      302                  45                1654   
2     359      234                  39                1335   
3     100      468                  31                1017   
4      68      369                  23                 748   
..    ...      ...                 ...                 ...   
71    107        0                  38                1112   
72     59        0                  28                 761   
73      0        0                   4                 234   
74      

In [20]:
table = tables[0]

In [21]:
table.head()

Airport    Terminal      Date         Hour       U.S. Citizen  \
  Airport    Terminal      Date         Hour       U.S. Citizen   
  Airport    Terminal      Date         Hour Average Wait  Time   
0     JFK  Terminal 1  8/7/2023  1000 - 1100                 32   
1     JFK  Terminal 1  8/7/2023  1100 - 1200                 41   
2     JFK  Terminal 1  8/7/2023  1300 - 1400                 31   
3     JFK  Terminal 1  8/7/2023  1400 - 1500                 41   
4     JFK  Terminal 1  8/7/2023  1500 - 1600                 61   

                   Non U.S. Citizen                               All  \
                   Non U.S. Citizen                        Wait Times   
  Max Wait  Time Average Wait  Time Max Wait  Time Average Wait  Time   
0             53                 29             52                 30   
1             91                 90            165                 72   
2            102                 97            153                 73   
3            138                110            179                 94   
4            148                134            171                107   

                  ...                                                       \
                  ... Number Of Passengers Time Interval                     
  Max Wait  Time  ...                              16-30 31-45 46-60 61-90   
0             53  ...                                146   270    82     0   
1            165  ...                                207   319   208   300   
2            153  ...                                211   322    64    61   
3            179  ...                                 87   204    77    35   
4            171  ...                                 37    57    74   120   

                                                                               \
                  Unnamed: 17_level_1 Unnamed: 18_level_1 Unnamed: 19_level_1   
  91-120 120 plus            Excluded               Total             Flights   
0      0        0                  19                 624                   2   
1    206      302                  45                1654                   5   
2    359      234                  39                1335                   5   
3    100      468                  31                1017                   4   
4     68      369                  23                 748                   2   

                       
  Unnamed: 20_level_1  
  Unnamed: 20_level_2  
0                 NaN  
1                 NaN  
2                 NaN  
3                 NaN  
4                 NaN  

[5 rows x 21 columns]

In [23]:
table.columns.values

array([('Airport', 'Airport', 'Airport'),
       ('Terminal', 'Terminal', 'Terminal'), ('Date', 'Date', 'Date'),
       ('Hour', 'Hour', 'Hour'),
       ('U.S. Citizen', 'U.S. Citizen', 'Average Wait  Time'),
       ('U.S. Citizen', 'U.S. Citizen', 'Max Wait  Time'),
       ('Non U.S. Citizen', 'Non U.S. Citizen', 'Average Wait  Time'),
       ('Non U.S. Citizen', 'Non U.S. Citizen', 'Max Wait  Time'),
       ('All', 'Wait Times', 'Average Wait  Time'),
       ('All', 'Wait Times', 'Max Wait  Time'),
       ('All', 'Number Of Passengers Time Interval', '0-15'),
       ('All', 'Number Of Passengers Time Interval', '16-30'),
       ('All', 'Number Of Passengers Time Interval', '31-45'),
       ('All', 'Number Of Passengers Time Interval', '46-60'),
       ('All', 'Number Of Passengers Time Interval', '61-90'),
       ('All', 'Number Of Passengers Time Interval', '91-120'),
       ('All', 'Number Of Passengers Time Interval', '120 plus'),
       ('All', 'Unnamed: 17_level_1', 'Excluded'),

### We clean up the columns and consolidate in one row

In [24]:
table.columns = ['_'.join(col) for col in table.columns.values]

In [25]:
table.rename(columns = {'Airport_Airport_Airport': 'Airport', 'Terminal_Terminal_Terminal':'Terminal', 'Date_Date_Date':'date', 'Hour_Hour_Hour':'hour' }, inplace = True)

In [26]:
table.rename(columns = {'U.S. Citizen_U.S. Citizen_Average Wait  Time': 'US Citizen Avg', 'U.S. Citizen_U.S. Citizen_Max Wait  Time':'US Citizen Max', 'Non U.S. Citizen_Non U.S. Citizen_Average Wait  Time':'Non-Citizen Avg', 'Non U.S. Citizen_Non U.S. Citizen_Max Wait  Time': 'Non-Citizen Max' }, inplace = True)

In [27]:
table.rename(columns = {'All_Wait Times_Average Wait  Time': 'Avg Wait Time', 'All_Wait Times_Max Wait  Time':'Max Wait Time', 'All_Unnamed: 18_level_1_Total':'Total Passengers', 'All_Unnamed: 19_level_1_Flights': 'Total Flights' }, inplace = True)

In [28]:
table = table[['Airport', 'Terminal', 'date', 'hour','US Citizen Avg', 'US Citizen Max', 'Non-Citizen Avg','Non-Citizen Max','Avg Wait Time', 'Max Wait Time','Total Passengers','Total Flights' ]]

In [30]:
table.to_csv("cbp-today.csv", index=False)

In [38]:
term1 = table[table["Terminal"].str.contains("1")].reset_index(inplace=False)

In [39]:
term1 = term1.drop('index', axis =1)

In [40]:
reduced1 = term1[['hour','US Citizen Avg', 'Non-Citizen Avg']]

In [41]:
reducedtransposed1 = reduced1.transpose()

In [42]:
reducedtransposed1.columns = reducedtransposed1.iloc[0]

In [43]:
reducedtransposed1 = reducedtransposed1.drop(index= "hour")

In [44]:
reducedtransposed1 

hour,1000 - 1100,1100 - 1200,1300 - 1400,1400 - 1500,1500 - 1600,1600 - 1700,1700 - 1800,1800 - 1900,1900 - 2000,2000 - 2100,2100 - 2200,2200 - 2300,2300 - 0000
US Citizen Avg,32,41,31,41,61,37,51,25,18,17,28,20,19
Non-Citizen Avg,29,90,97,110,134,122,120,55,46,70,50,26,25


In [34]:
reducedtransposed1.to_csv("term1_us_nonus.csv")

### Terminal 4:

In [36]:
term4 = table[table["Terminal"].str.contains("4")].reset_index(inplace=False)

In [37]:
term4 = term4.drop('index', axis =1)
reduced4 = term4[['hour','US Citizen Avg', 'Non-Citizen Avg']]
reducedtransposed4 = reduced4.transpose()
reducedtransposed4.columns = reducedtransposed4.iloc[0]
reducedtransposed4 = reducedtransposed4.drop(index= "hour")

In [43]:
reducedtransposed4.to_csv("term4_us_nonus.csv")

### Terminal 5: 

In [40]:
term5 = table[table["Terminal"].str.contains("5")].reset_index(inplace=False)

In [41]:
term5 = term5.drop('index', axis =1)
reduced5 = term5[['hour','US Citizen Avg', 'Non-Citizen Avg']]
reducedtransposed5 = reduced5.transpose()
reducedtransposed5.columns = reducedtransposed5.iloc[0]
reducedtransposed5 = reducedtransposed5.drop(index= "hour")

In [45]:
reducedtransposed5.to_csv("term5_us_nonus.csv")

### Terminal 7: 

In [47]:
term7 = table[table["Terminal"].str.contains("7")].reset_index(inplace=False)

In [48]:
term7 = term7.drop('index', axis =1)
reduced7 = term7[['hour','US Citizen Avg', 'Non-Citizen Avg']]
reducedtransposed7 = reduced7.transpose()
reducedtransposed7.columns = reducedtransposed7.iloc[0]
reducedtransposed7 = reducedtransposed7.drop(index= "hour")

In [49]:
reducedtransposed7.to_csv("term7_us_nonus.csv")

### Terminal 8: 

In [55]:
term8 = table[table["Terminal"].str.contains("8")].reset_index(inplace=False)

In [56]:
reduced8 = term8[['hour','US Citizen Avg', 'Non-Citizen Avg']]
reducedtransposed8 = reduced8.transpose()
reducedtransposed8.columns = reducedtransposed8.iloc[0]
reducedtransposed8 = reducedtransposed8.drop(index= "hour")

In [57]:
reducedtransposed8.to_csv("term8_us_nonus.csv")

In [58]:
ls

README.md              original_cbp.ipynb     term7_us_nonus.csv
action.yml             term1_us_nonus.csv     term8_us_nonus.csv
cbp1.ipynb             term4_us_nonus.csv
jfkterm8_us_nonus.csv  term5_us_nonus.csv


In [59]:
await browser.close()